<a href="https://colab.research.google.com/drive/1tElT3FhddabqCyJKFH27rhnyXJTqrzke?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K Nearest Neighbors (KNN)

## Introduction

The k Nearest Neighbors method (kNN) is a very popular classification method, also sometimes used in regression tasks. It's one of the most understandable approaches to classification. Intuitively, the essence of the method is: look at the neighbors; the predominant ones indicate what you are. Formally, the basis of the method is the compactness hypothesis: if the distance metric between examples is introduced successfully, then similar examples are much more likely to be in the same class than in different ones.

<img src='https://hsto.org/web/68d/a45/6f0/68da456f00f8434e87628dbe7e3f54a7.png' width=600>

 For the classification of each object in the test dataset, it is necessary to sequentially perform the following operations:
   * Calculate the distance to each of the objects in the training dataset.
   * Select objects from the training dataset to which the distance is minimal.
   * The class of the classified object is the class most frequently occurring among the k nearest neighbors.

We will work with a subset of the [Covertype dataset from the UCI repository](http://archive.ics.uci.edu/ml/datasets/Covertype). There are 7 different classes available. Each object is described by 54 features, 40 of which are binary. The data description is available at the link.

## Preprocessing

In [ ]:
import pandas as pd
import numpy as np

Press [here](https://drive.google.com/file/d/1Z39LNnF4lOj4iT48YDYFZNPDBglMd0TZ/view?usp=sharing) to download the dataset.

In [ ]:
all_data = pd.read_csv('datasets/forest_dataset.csv')
all_data.head()

In [ ]:
all_data.shape

Extract the class labels into a variable `labels`, and the feature descriptions into a variable `feature_matrix`. Since the data is numerical and does not have missing values, we will convert it to `numpy` format using the `.values` method.

In [ ]:
labels = all_data[all_data.columns[-1]].values
feature_matrix = all_data[all_data.columns[:-1]].values

## Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

We will now work with all 7 types of cover (the data is already in the variables `feature_matrix` and `labels` if you haven't redefined them). Divide the sample into training and test sets using the `train_test_split` method.


In [ ]:
train_feature_matrix, test_feature_matrix, train_labels, test_labels = train_test_split(
    feature_matrix, labels, test_size=0.2, random_state=42)

The main objects in `sklearn` are the so-called `estimators`. They are divided into **classifiers** and **regressors**.

As examples of models, you can consider the classifiers
[k Nearest Neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) and
[logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

All models in `sklearn` must have at least 2 methods  -- `fit` and `predict`.
The `fit(X, y)` method is responsible for training the model and takes as input a training sample in the form of a *feature matrix* $X$ and a *response vector* $y$.

After `fit`, you can now call the `predict(X)` method on the trained model, which will return predictions of this model on all objects from the matrix $X$ in the form of a vector.

You can call `fit` on the same model several times; each time it will be trained anew on the given dataset.

Also, models have *hyperparameters*, which are usually set when creating the model.

Let's see all this on the example of logistic regression.


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(C=1)
clf.fit(train_feature_matrix, train_labels)
y_pred = clf.predict(test_feature_matrix)

Now, we would like to measure the quality of our model. For this, we can use the `score(X, y)` method, which will calculate some error function on the sample $X, y$, but exactly which one depends on the model. Alternatively, one of the functions from the `metrics` module can be used, for example, [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), which, as the name suggests, will calculate the accuracy of predictions for us.


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_labels, y_pred)

Finally, the last thing we would like to mention is grid search for hyperparameter tuning. Since models have many hyperparameters that can be changed, and the quality of the model significantly depends on these hyperparameters, it would be desirable to find the best parameters in this sense. The simplest way to do this is to just iterate over all possible variants within reasonable limits.

This can be done using the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class, which conducts a search (search) across a grid (grid) and calculates the quality of the model using cross-validation (CV).

For example, in logistic regression, you can change the `C` and `penalty` parameters. Let's do that. Keep in mind that the search may take a long time. Refer to the documentation for the meaning of the parameters.


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = LogisticRegression(solver='saga')

# init GridSearchCV with parameters
param_grid = {
    'C': np.arange(1, 5),
    'penalty': ['l1', 'l2'],
}

search = GridSearchCV(clf, param_grid, n_jobs=-1, cv=5, refit=True, scoring='accuracy')

search.fit(feature_matrix, labels)

print(search.best_params_)

In this case, the search iterates over all possible pairs of values for `C` and `penalty` from the specified sets.


In [ ]:
accuracy_score(labels, search.best_estimator_.predict(feature_matrix))

Note that we pass the entire dataset to GridSearchCV, not just its training part. This can be done because the search still uses cross-validation. However, sometimes a *validation* part is still separated from the dataset, as the hyperparameters might overfit to the sample during the search process.

In the tasks, you will need to repeat this for the k Nearest Neighbors method.

# Homework

The quality of classification/regression by the k Nearest Neighbors method depends on several parameters:
* the number of neighbors `n_neighbors`
* the distance metric between objects `metric`
* the weights of neighbors (the neighbors of the test example can enter with different weights, for example, the further the example, the less its "voice" is taken into account) `weights`


Train the `KNeighborsClassifier` on the dataset.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf = # Your code here
# ...
# Your code here

Let's select the parameters of our model:

* Iterate over the grid from `1` to `10` for the number of neighbors parameter
* Also, try using different metrics: `['manhattan', 'euclidean']`
* Try using different weight calculation strategies: `[‘uniform’, ‘distance’]`

In [ ]:
from sklearn.model_selection import GridSearchCV
params = ...

clf_grid = GridSearchCV(clf, params, cv=5, scoring='accuracy', n_jobs=-1)
# Train the classifier
# ...

Let's output the best parameters

In [ ]:
clf_grid.best_params_

Using the optimal number of neighbors found, calculate the probabilities of belonging to classes for the test sample (`.predict_proba`).

In [ ]:
optimal_clf = # ...
# Training ...
pred_prob = # ...

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

unique, freq = np.unique(test_labels, return_counts=True)
freq = list(map(lambda x: x / len(test_labels),freq))

pred_freq = pred_prob.mean(axis=0)
plt.figure(figsize=(10, 8))
plt.bar(range(1, 8), pred_freq, width=0.4, align="edge", label='prediction')
plt.bar(range(1, 8), freq, width=-0.4, align="edge", label='real')
plt.ylim(0, 0.54)
plt.legend()
plt.show()

## Conclusion

What is the quality of the model on the test sample? Compare it with the quality of logistic regression. Which model is better? Why? What are the pros and cons of the k Nearest Neighbors method? Plot the ROC curve for the k Nearest Neighbors method. Calculate the area under the ROC curve (AUC-ROC).